# Functions

The content material is taken from https://techytok.com/from-zero-to-julia/ 

**Table of Contents**
 - [Defining Functions](#Defining-Functions)
 - [Void Functions](#Void-Functions)
 - [Anonymous Functions](#Anonymous-Functions)
 - [Optional and Keyword Arguments](#Optional-and-Keyword-Arguments)
 - [Documenting Functions](#Documenting-Functions)
 - [Conclusion](#Conclusion)
            
Functions are the main building blocks in Julia. Every operation on variables and other elements is performed through functions, even the mathematical operators (e.g. `+`) are functions in an infix form.


## Defining Functions

A typical function looks like this: **NOTE: Julia doesn't enforce indentation as Python**

In [ ]:
function plusTwo(x)
    return x + 2
end

println( "5 + 2 = ", plusTwo(5) )
println( "1 + 2 = ", plusTwo(1) )

# we can restrict types
function plusTwo(x::Int32)::Int32
    return x+2
end

println( "5 + 2 = ", plusTwo(5) )
println( "1 + 2 = ", plusTwo(1) )

Although the previous is the most common way to write functions, it is sometimes convenient to use the inline version (think of the REPL):

In [ ]:
plusTwo(x) = x+2

println( "2 + 2 = ", plusTwo(2) )
println( "3 + 2 = ", plusTwo(3) )

# we can restrict types
plusTwo(x::Real) = x+2

println( "2 + 2 = ", plusTwo(2) )
println( "3 + 2 = ", plusTwo(3) )


A few important things:
- Functions are defined with the keyword **function** and always finalized with **end**. 
- Julia **doesn't have member functions** ( foo.bar() ), functions can be scoped with modules (think Fortran).
- Function arguments and return types are optional, but highly recommended
- Function arguments can limit types using Abstract* types (see example below)
- Functions can have multiple signatures (overloaded), it's the basis for multiple dispatch
- Function arguments are *"passed by reference"*. Just like in Fortran, single values won't be modified, but array contents will.
- It's a convention that if a function modifies its arguments it ends with `!`
- The dot (`.`) operator is syntactic sugar for element by element operations (vectorization), see [article](https://julialang.org/blog/2017/01/moredots/). It's also called broadcasting.

In [ ]:
# modifies v
function add!(v::AbstractArray{},c)
    if size(v,1) >= 1      # bounds check
        v[1] = v[1] + c
    end
    return
end

function run()
  v = [1]
  add!(v,2)
  println("v is now: ", v)   
  return
end

run()


function add!(v::AbstractArray{}, c)
    # dot operates element-by-element (think of dot product)
    # must be next to operator (=, +)
    v .= v .+ c  
end

function run()
  v = [1,2,3]
  add!(v,1) 
  println("v is now: ", v)   
  return
end

run()


## Void Functions

Julia has a unique signature for all functions. `void` functions (`subroutines` in Fortran), those that don't return anything. Functions may also take no arguments and return no value, if needed, for example we can create a function which prints a string:

In [ ]:
function say_hi()
    println("Hello from ORNL!")
    return
end

say_hi()

If the function returns no value `return` can be omitted. In general, ~it is also possible to omit the return statement even in regular functions and Julia will return the last computed value~. **NOTE: bad idea**. However, in my opinion, in this case it is better to return explicitly a value, for clarity’s sake.

In [ ]:
function say_hi()
    println("Hello from ORNL without return!")
end

say_hi()

## Anonymous Functions

It is also possible to create anonymous functions (like lambdas in Python or C++) using the following structure:

In [ ]:
plusThree = x -> x+3

println( "5 + 3 = ", plusThree(5) )

**NOTE: It is not recommended to use anonymous functions unless they are really simple.** It is generally better to write functions using the `function/end` keywords unless you need to write a small wrapper around another function and pass it to a third function. (Think of lambda functions in C++ and Python)

```
The following example is a bit more advanced and it includes topics which we have not yet studied, like integrals and how to install and use an external package. You can always come back later to read this example as it is not fundamental at this point of the course but you may find it useful one day. The take home message is never use anonymous functions unless you know what you are doing.
```

For example, let’s consider a function `f` of three variables `x`, `y` and `z`. Let’s suppose we want to fix two variables (y and z) and integrate f over x, we could do it with:

In [ ]:
import QuadGK

f(x,y,z) = (x^2 + 2y)*z

result = QuadGK.quadgk(x->f(x,42,4), 3, 4)
println("Integration and error usgin Gauss quadrature: ", result)

or use an intermediate function

In [ ]:
f(x,y,z) = (x^2 + 2y)*z
g(x) = f(x,42,4)

result = QuadGK.quadgk(g, 3, 4)
println("Integration and error using Gauss quadrature: ", result)

## Optional and Keyword Arguments

### Optional arguments
Sometimes a parameter may have a default value which can be specified so that the user doesn’t need to always type it. For example let’s write a function which converts our “weight” as measured on Earth (in kg) to the one measured on another planet.

In [ ]:
function myWeight(weightOnEarth, g=9.81)
    return weightOnEarth*g/9.81
end

println( "My weight on Earth: ", myWeight(90), " kg"  )
println( "My weight on Mars:  ", myWeight(90, 3.721), " kg" )

As the name suggests positional arguments must be used in the right order, we cannot specify g before `weightOnEarth` and, as opposed to other languages like Python, in Julia we cannot change the order of the arguments even if we specify the name of the parameter. If we want optional arguments with no fixed position, we need to use **keyword arguments**.

### Keyword arguments 
Keyword arguments are separated from positional arguments by a semicolon ; and must always be addressed by their name, although their order is irrelevant. They can be either optional and not, but usually we use keyword arguments for optional parameters.

In [ ]:
# positional arguments ; keyword arguments
function my_long_function(a, b=2; c, d=3)
    return a + b + c + d
end

Here `a` and `b` are positional arguments, while `c` and `d` are keyword arguments. Keyword arguments must always be specified either by default or explicitly.

In [ ]:
# Usage Examples:

# a=1, b=2 (default), c=3, d=3 (default)
println( my_long_function(1, c=3) )

# a=1, b=2, c=3, d=3 (default)
println( my_long_function(1, 2, c=3) )

# a=1, b=2, c=3, d=5, order doesn't matter for keyword arguments
println( my_long_function(1, 2, d=5, c=3) )

In [ ]:
# Error, c is not defined, Keyword argument must be defined
my_long_function(1, 2, d=5)

**Tip**: prefer positional arguments for performance (function called several times) as it's closer to underlying libraries (C, Fortran, C++)

## Documenting Functions

Julia supports function API documentation using Markdown, similar to Python docstring. Function documentation is done above a function using the """ """ opening and closing symbols. While the macro `@doc` (or the `?function_name` in the REPL) exposes the documentation information if available.

In [ ]:
# Example: show built-in sin documentation
@doc sin

In [ ]:
"""
add(x,y)

Compute the sum of 2 numbers `x + y`
    
# Arguments:
- x: input any number
- y: input any number

# Examples
```
julia> add(2,4)
6
```
"""
function add(x,y)
    return x+y
end

@doc add

## Conclusion

In this lesson we learned how to define functions and use positional as well as keyword arguments.

**Questions:**

1. How many way exist to define a function in Julia?
2. Can functions be member functions in Julia?
3. What's the difference between Optional and Keyword arguments?
4. How do you document your function API in Julia?